# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import zipfile

# Load the data

In [ ]:
# Unzip "Predios_Completos":
zip_path = "Predios_Completos.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()
    print("Archivos extraídos:", zip_ref.namelist())

In [2]:
data_all_properties = gpd.read_file("predios_completos.gpkg")
data_all_properties.head(3)

,gid,id,distrito,shape_leng,shape_area,UNuso,layer,path,geometry
0,119895,95475.0,2.0,54.193112,130.922608,Habitacional,USOS_DE_SUELO_Z2,/Users/eileenyuen/Downloads/OneDrive_1_30-4-20...,"MULTIPOLYGON (((672237.33 2295207.029, 672235...."
1,119896,471562.0,2.0,47.842126,112.073148,Habitacional,USOS_DE_SUELO_Z2,/Users/eileenyuen/Downloads/OneDrive_1_30-4-20...,"MULTIPOLYGON (((669537.234 2295451.983, 669520..."
2,119897,53588.0,2.0,54.517475,144.209492,Habitacional,USOS_DE_SUELO_Z2,/Users/eileenyuen/Downloads/OneDrive_1_30-4-20...,"MULTIPOLYGON (((671188.527 2293497.949, 671187..."


# Checking the "UNuso" variable

In [3]:
data_all_properties["UNuso"].unique()

array(['Habitacional', 'Mixto', 'Baldio', None,
       'Espacios_verdes_abiertos_y_recreativos',
       'Granjas_huertos_y_servicios', 'Sin_dato', 'Comercio',
       'Equipamiento_institucional', 'Agropecuario',
       'Manufacturas_e_industria', 'Infraestructura',
       'Servicios_a_la_industria', 'Forestal', 'Alojamiento_temporal',
       'Servicios', 'Area_libre', 'Comercios', 'Industria',
       'Servicios_a_la_industria_y_al_comercio', 'Turistico_hotelero',
       'Granjas_y_huertos', 'Instalaciones_especiales',
       'Actividad_silvestre', 'Sin_uso', 'Actividades_extractivas',
       'Comercial', 'Espacios_verdes', 'Turistico_campestre',
       'Manufacturas_menores', 'Turistico_ecologico', 'Piscicola',
       'Areas_agricolas', 'Servicio',
       'Instalaciones_especiales_e_infraestructura_urbana', 'Sin_datos',
       'Area_silvestre', 'Areas_silvestres', 'Area_natural_protegida',
       'Espacio_verde', 'Area_natural_protegida_nixticuil',
       'Sin_datos_o_baldio', 'Cuerpo_

### There are many labels that have smilar names to each other. So the ones that share similarity in their names, we are going to join them on a single label

In [13]:
# Ver todas las categorías únicas y su frecuencia
print(data_all_properties["UNuso"].value_counts(dropna=False))

Habitacional                                         360723
Baldio                                                52839
Agropecuario                                          10720
Mixto                                                  7178
Comercio                                               5116
Servicio                                               3580
Sin_uso                                                3230
Actividades_extractivas                                2458
Espacios_verdes_abiertos_y_recreativos                 2372
None                                                   2244
Servicios                                              1969
Area_natural_protegida                                 1908
Equipamiento_institucional                             1791
Industria                                              1376
Comercial                                              1195
Espacios_verdes                                        1175
Areas_agricolas                         

#### Checking the frequency of certain labels (and also their complete names) that have similar meaning

In [4]:
# Encontrar todas las categorías que contengan "comer"
comercial_pattern = [x for x in data_all_properties["UNuso"].unique() 
                    if x and any(word in str(x).lower() for word in ['comer', 'comerc'])]
print(comercial_pattern)

['Comercio', 'Comercios', 'Servicios_a_la_industria_y_al_comercio', 'Comercial']


In [5]:
# Encontrar todas las categorías que contengan "comer"
services_pattern = [x for x in data_all_properties["UNuso"].unique() 
                    if x and any(word in str(x).lower() for word in ['serv', 'servi'])]
print(comercial_pattern)

['Comercio', 'Comercios', 'Servicios_a_la_industria_y_al_comercio', 'Comercial']


In [6]:
# Encontrar todas las categorías que contengan "comer"
turism_pattern = [x for x in data_all_properties["UNuso"].unique() 
                    if x and any(word in str(x).lower() for word in ['turi'])]
print(comercial_pattern)

['Comercio', 'Comercios', 'Servicios_a_la_industria_y_al_comercio', 'Comercial']


In [7]:
# Encontrar todas las categorías que contengan "comer"
green_pattern = [x for x in data_all_properties["UNuso"].unique() 
                    if x and any(word in str(x).lower() for word in ['verde'])]
print(comercial_pattern)

['Comercio', 'Comercios', 'Servicios_a_la_industria_y_al_comercio', 'Comercial']


In [8]:
# Encontrar todas las categorías que contengan "comer"
natural_area_pattern = [x for x in data_all_properties["UNuso"].unique() 
                    if x and any(word in str(x).lower() for word in ['natural'])]
print(comercial_pattern)

['Comercio', 'Comercios', 'Servicios_a_la_industria_y_al_comercio', 'Comercial']


In [11]:
# Diccionario de mapeo para consolidar categorías
mapeo = {
    # Consolidación de categorías comerciales
    'Comercio': 'Comercial',
    'Comercios': 'Comercial',
    
    # Consolidación de categorías de servicios
    'Servicios': 'Servicio',
    'Servicios_a_la_industria': 'Servicio_a_la_industria',
    'Servicios_a_la_industria_y_al_comercio': 'Servicio_a_la_industria',
    
    # Consolidación de categorías de espacios verdes
    'Espacios_verdes': 'Espacio_verde',
    'Espacios_verdes_abiertos_y_recreativos': 'Espacio_verde',
    
    # Consolidación de categorías de granjas/huertos
    'Granjas_huertos_y_servicios': 'Granjas_y_huertos',
    'Agropecuario': 'Granjas_y_huertos',
    'Areas_agrícolas': 'Granjas_y_huertos',
    'Piscicola': 'Granjas_y_huertos',
    'Forestal': 'Granjas_y_huertos',
    
    
    # Consolidación de categorías silvestres
    'Area_silvestre': 'Areas_silvestres',
    'Actividad_silvestre': 'Areas_silvestres',
    
    # Consolidación de categorías de sin dato
    'Sin_dato': 'Sin_datos',

    # Baldio
    'Sin_datos_o_baldio': 'Baldio',
    
    # Consolidación de categorías de instalaciones
    'Instalacion_especial': 'Instalaciones_especiales',
    'Instalaciones_especiales_e_infraestructura_urbana': 'Instalaciones_especiales',
    
    # Consolidación de categorías manufactureras
    'Manufacturas_e_industria': 'Manufacturas', # Renombrar a "manufacturas"
    'Manufacturas_domiciliarias': 'Manufacturas',
    'Manufacturas_menores': 'Manufacturas',
    
    # Consolidación de categorías turísticas
    'Turistico_hotelero': 'Turistico',
    'Turistico_campestre': 'Turistico',
    'Turistico_ecologico': 'Turistico',

    # Consolidación de áreas naturales protegidas
    'Area_natural_protegida_nixticuil': 'Area_natural_protegida'
}

In [18]:
# Usando el mismo diccionario de mapeo que definimos antes
# Mantiene los valores originales para los casos donde no hay coincidencia en el mapeo
data_all_properties["UNuso_consolidado"] = data_all_properties["UNuso"].map(mapeo).fillna(data_all_properties["UNuso"]) 

# Verificar los resultados
print(data_all_properties["UNuso_consolidado"].unique())

['Habitacional' 'Mixto' 'Baldio' None 'Espacio_verde' 'Granjas_y_huertos'
 'Sin_datos' 'Comercial' 'Equipamiento_institucional' 'Manufacturas'
 'Infraestructura' 'Servicio_a_la_industria' 'Alojamiento_temporal'
 'Servicio' 'Area_libre' 'Industria' 'Turistico'
 'Instalaciones_especiales' 'Areas_silvestres' 'Sin_uso'
 'Actividades_extractivas' 'Areas_agricolas' 'Area_natural_protegida'
 'Cuerpo_de_agua' 'Equipamiento']


In [21]:
data_cleaned = data_all_properties.copy()
data_cleaned.columns.tolist()

['gid',
 'id',
 'distrito',
 'shape_leng',
 'shape_area',
 'UNuso',
 'layer',
 'path',
 'geometry',
 'UNuso_consolidado']

In [31]:
data_cleaned.drop(columns = ['UNuso'], inplace = True) # Eliminar la antigua columna con los datos desagregados
data_cleaned.rename(columns = {'UNuso_consolidado': 'UNuso'}, inplace = True)

In [32]:
data_cleaned.columns.to_list() # Comprobar resultado

['gid',
 'id',
 'distrito',
 'shape_leng',
 'shape_area',
 'layer',
 'path',
 'geometry',
 'UNuso']

In [33]:
# Verificar datos faltantes y valores especiales
print(f"Nulos: {data_all_properties['UNuso'].isna().sum()}")
print(f"Valores 'Sin sin uso': {(data_all_properties['UNuso'] == 'Sin_uso').sum()}")

Nulos: 2244
Valores 'Sin sin uso': 3230
